In [29]:
import nltk
from nltk.tag import hmm
from nltk.probability import LaplaceProbDist, MLEProbDist, ConditionalFreqDist, ConditionalProbDist,LidstoneProbDist
import sys
import re

characters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',' ',',','.']
char_dict = {
    'a':0,
    'b':1,
    'c':2,
    'd':3,
    'e':4,
    'f':5,
    'g':6,
    'h':7,
    'i':8,
    'j':9,
    'k':10,
    'l':11,
    'm':12,
    'n':13,
    'o':14,
    'p':15,
    'q':16,
    'r':17,
    's':18,
    't':19,
    'u':20,
    'v':21,
    'w':22,
    'x':23,
    'y':24,
    'z':25,
    ' ':26,
    ',':27,
    '.':28
}

def preprocess(text):
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"!", "!", text)
    text = re.sub(r"\/", "", text)
    text = re.sub(r"\^", "^", text)
    text = re.sub(r"\+", "+", text)
    text = re.sub(r"\-", "-", text)
    text = re.sub(r"\=", "=", text)
    text = re.sub(r'[0-9\s]',"=",text)
    text = re.sub(r"'", "=", text)
    text = re.sub(r"^https?:\/\/.*[\r\n]*","", text)
    text = re.sub(r'[^.,a-zA-Z0-9\s]', ' ',text)
    text = re.sub(r'  ',"",text)
    return text




LAPLACE = False
LM = False

if "-laplace" in sys.argv:
    LAPLACE = True
if "--lm" in sys.argv:
    LM = True

directory = sys.argv[-1]


file1 = open("a2data/cipher2/train_plain.txt","r")
file1_train_plain = file1.readlines()
file1_train_plain = [line.lower().strip('\n') for line in file1_train_plain]
file1_train_plain = [list(line) for line in file1_train_plain]
file1.close()

file1 = open("a2data/cipher2/train_cipher.txt","r")
file1_train_cipher = file1.readlines()
file1_train_cipher = [line.lower().strip('\n') for line in file1_train_cipher]
file1_train_cipher = [list(line) for line in file1_train_cipher]
file1.close()

for_tagging = []
for i in range(len(file1_train_cipher)):    #(cipher,plaintext)
    partial_list = []
    for j in range(len(file1_train_cipher[i])):
            partial_list.append((file1_train_cipher[i][j],file1_train_plain[i][j]))
    for_tagging.append(partial_list)

#for_tagging = [tup for tup in for_tagging]
#print(for_tagging)


In [30]:
def estimator(fd, bins):
    return LaplaceProbDist(fd,bins)


LAPLACE = False
LM = True

new_file = open("a2data/TheStory.txt","r",encoding="utf8")
new_file = new_file.readlines()
#new_file = []

# addresses = ["a2data/cipher1/train_plain.txt","a2data/cipher1/test_plain.txt","a2data/cipher2/train_plain.txt",\
#              "a2data/cipher2/test_plain.txt","a2data/cipher3/train_plain.txt","a2data/cipher3/test_plain.txt"]
# # append existing corpus to the end
# for address in addresses:
#     file1 = open(address,"r")
#     file1_train_plain = file1.readlines()
#     file1_train_plain = [line.lower().strip('\n') for line in file1_train_plain]
#     #file1_train_plain = [list(line) for line in file1_train_plain]
#     file1.close()
#     new_file = new_file + file1_train_plain

new_file = [line.lower().strip('\n').strip() for line in new_file]
new_file = [line for line in new_file if len(line)>1]
    
new_corpus = [preprocess(line.strip()) for line in new_file if len(line)>1]

final_corpus1 = ""
for line in new_corpus:
    final_corpus1 += line
    
final_corpus2 = final_corpus1[1:]
final_pairs=[]
for i in range(len(final_corpus1)-1):
    final_pairs.append((final_corpus1[i],final_corpus2[i]))




trainer = hmm.HiddenMarkovModelTrainer()
tagger = 0 
if LAPLACE:
    tagger = trainer.train_supervised(for_tagging,estimator)
    if LM:
        cfdist = ConditionalFreqDist(tup for tup in final_pairs)
        cpd = ConditionalProbDist(cfdist,estimator, 29)
        attribute = tagger.__getattribute__("_transitions")
        for (cond, prob_dist) in cpd.items():
            new_fd = prob_dist.freqdist() + attribute[cond].freqdist()
            if type(prob_dist) is LaplaceProbDist:
                cpd[cond] = LaplaceProbDist(new_fd, prob_dist._bins)
            elif type(prob_dist) is MLEProbDist:
                cpd[cond] = MLEProbDist(new_fd)
        tagger.__getattribute__('_transitions').update(cpd)
elif LM:
    tagger = trainer.train_supervised(for_tagging)
    cfdist = ConditionalFreqDist(tup for tup in final_pairs)
    cpd = ConditionalProbDist(cfdist,estimator, 29)
    attribute = tagger.__getattribute__("_transitions")
    for (cond, prob_dist) in cpd.items():
        new_fd = prob_dist.freqdist() + attribute[cond].freqdist()
        if type(prob_dist) is LaplaceProbDist:
            cpd[cond] = LaplaceProbDist(new_fd, prob_dist._bins)
        elif type(prob_dist) is MLEProbDist:
            cpd[cond] = MLEProbDist(new_fd)
    tagger.__getattribute__('_transitions').update(cpd)
else:
    tagger = trainer.train_supervised(for_tagging)

In [31]:
file1 = open("a2data/cipher2/test_plain.txt","r")
file1_test_plain = file1.readlines()
file1_test_plain = [line.lower().strip('\n') for line in file1_test_plain]
file1_test_plain = [list(line) for line in file1_test_plain]
file1.close()

file1 = open("a2data/cipher2/test_cipher.txt","r")
file1_test_cipher = file1.readlines()
file1_test_cipher = [line.lower().strip('\n') for line in file1_test_cipher]
file1.close()

tagged_result = []

for i in range(len(file1_test_cipher)):
    tagged_list = tagger.tag(file1_test_cipher[i])
    tagged_result.append(tagged_list)

file1_test_plain = [list(line) for line in file1_test_plain]

char_counter = 0
correct_counter = 0
decipherment = []
for i in range(len(tagged_result)):
    for j in range(len(tagged_result[i])):
        char_counter += 1
        decipherment.append(tagged_result[i][j][1])
        if tagged_result[i][j][1] == file1_test_plain[i][j]:
            correct_counter += 1

# print(correct_counter)
# print(char_counter)
print(decipherment)
print("The decipherment accuracy:",correct_counter / char_counter)

['h', ' ', 'h', 'i', 'a', 'd', 'e', 'n', ' ', 'm', 'a', 'r', 'k', 'o', 'u', ' ', 'm', 'o', 'd', 'e', 'l', ' ', 'i', 's', ' ', 'o', 'n', 'e', ' ', 'i', 'n', ' ', 'w', 'h', 'i', 'c', 'h', ' ', 'w', ' ', 't', ' ', 'o', 'v', 'e', 'e', 'r', 'v', 'e', ' ', 'd', ' ', 's', 'e', 'r', 't', 'e', 'n', 'c', 'e', ' ', 'o', 'f', ' ', 'e', 'm', 'i', 's', 's', 'i', 'o', 'n', 's', ',', ' ', 'b', 'u', 't', ' ', 'd', ' ', 'o', 'n', ' ', 't', 'o', 'n', 'g', 'o', 'w', ' ', 't', 'h', 'e', ' ', 's', 'e', 'q', 'u', 'e', 'n', 's', 'e', ' ', 'o', 'f', ' ', 's', 't', 'a', 'l', 'e', 's', ' ', 't', 'h', 'e', ' ', 'm', ' ', 'd', 'e', 'l', ' ', 'w', 'e', 'n', 't', ' ', 't', 'h', 'r', 'o', 'u', 'n', 'g', ' ', 't', 'o', ' ', 'h', 'e', 'n', 'e', 'r', 'a', 't', 'e', ' ', 't', 'h', 'e', ' ', 'e', 'm', 'i', 's', 's', 'i', 'o', 'n', 's', '.', 'a', 'n', 'a', 'l', 'y', 's', 'e', 's', ' ', 'o', 'f', ' ', 'h', 'i', 'd', 'd', 'e', 'n', ' ', 'm', 'a', 'r', 'k', ' ', 'v', 'o', 'm', ' ', 'd', 'e', 'l', 's', ' ', 's', 'e', 'e', 'x',

In [116]:
###### new corpus
import re
characters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',' ',',','.']
char_dict = {
    'a':0,
    'b':1,
    'c':2,
    'd':3,
    'e':4,
    'f':5,
    'g':6,
    'h':7,
    'i':8,
    'j':9,
    'k':10,
    'l':11,
    'm':12,
    'n':13,
    'o':14,
    'p':15,
    'q':16,
    'r':17,
    's':18,
    't':19,
    'u':20,
    'v':21,
    'w':22,
    'x':23,
    'y':24,
    'z':25,
    ' ':26,
    ',':27,
    '.':28
}
def preprocess(text):
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"!", "!", text)
    text = re.sub(r"\/", "", text)
    text = re.sub(r"\^", "^", text)
    text = re.sub(r"\+", "+", text)
    text = re.sub(r"\-", "-", text)
    text = re.sub(r"\=", "=", text)
    text = re.sub(r'[0-9\s]',"=",text)
    text = re.sub(r"'", "=", text)
    text = re.sub(r"^https?:\/\/.*[\r\n]*","", text)
    text = re.sub(r'[^.,a-zA-Z0-9\s]', ' ',text)
    text = re.sub(r'  ',"",text)
    return text


new_file = open("a2data/TheStory.txt","r",encoding="utf8")
new_file = new_file.readlines()
#new_file = []

# addresses = ["a2data/cipher1/train_plain.txt","a2data/cipher1/test_plain.txt","a2data/cipher2/train_plain.txt",\
#              "a2data/cipher2/test_plain.txt","a2data/cipher3/train_plain.txt","a2data/cipher3/test_plain.txt"]
# # append existing corpus to the end
# for address in addresses:
#     file1 = open(address,"r")
#     file1_train_plain = file1.readlines()
#     file1_train_plain = [line.lower().strip('\n') for line in file1_train_plain]
#     #file1_train_plain = [list(line) for line in file1_train_plain]
#     file1.close()
#     new_file = new_file + file1_train_plain

new_file = [line.lower().strip('\n').strip() for line in new_file]
new_file = [line for line in new_file if len(line)>1]
    
new_corpus = [preprocess(line.strip()) for line in new_file if len(line)>1]

final_corpus1 = ""
for line in new_corpus:
    final_corpus1 += line
    
final_corpus2 = final_corpus1[1:]
final_pairs=[]
for i in range(len(final_corpus1)-1):
    final_pairs.append((final_corpus1[i],final_corpus2[i]))

In [119]:
final_pairs

[(' ', 't'),
 ('t', 'h'),
 ('h', 'e'),
 ('e', ' '),
 (' ', 's'),
 ('s', 't'),
 ('t', 'o'),
 ('o', 'r'),
 ('r', 'y'),
 ('y', ' '),
 (' ', 'c'),
 ('c', 'o'),
 ('o', 'n'),
 ('n', 't'),
 ('t', 'i'),
 ('i', 'n'),
 ('n', 'u'),
 ('u', 'e'),
 ('e', 's'),
 ('s', '.'),
 ('.', ' '),
 (' ', 'a'),
 ('a', ' '),
 (' ', 's'),
 ('s', 'e'),
 ('e', 'r'),
 ('r', 'i'),
 ('i', 'a'),
 ('a', 'l'),
 ('l', ' '),
 (' ', 'e'),
 ('e', 'n'),
 ('n', 'o'),
 ('o', 'v'),
 ('v', 'e'),
 ('e', 'l'),
 ('l', ' '),
 (' ', 'b'),
 ('b', 'y'),
 ('y', ' '),
 (' ', 'f'),
 ('f', 'e'),
 ('e', 'r'),
 ('r', 'd'),
 ('d', ' '),
 (' ', 'e'),
 ('e', 'g'),
 ('g', 'g'),
 ('g', 'a'),
 ('a', 'n'),
 ('n', 'w'),
 ('w', 'e'),
 ('e', 'l'),
 ('l', 'c'),
 ('c', 'o'),
 ('o', 'm'),
 ('m', 'e'),
 ('e', ' '),
 (' ', 't'),
 ('t', 'o'),
 ('o', ' '),
 (' ', 'h'),
 ('h', 'o'),
 ('o', 't'),
 ('t', 'e'),
 ('e', 'l'),
 ('l', ' '),
 (' ', 'r'),
 ('r', 'e'),
 ('e', 'a'),
 ('a', 'l'),
 ('l', ' '),
 (' ', 'd'),
 ('d', 'e'),
 ('e', 's'),
 ('s', 'e'),
 ('e', 'r'),

In [120]:
cfdist = ConditionalFreqDist(tup for tup in final_pairs)
cpd = ConditionalProbDist(cfdist,estimator, 29)
#trainer = hmm.HiddenMarkovModelTrainer()
attribute = tagger.__getattribute__("_transitions")
#print(attribute)
#tagger = trainer.train_unsupervised(final_pairs)

# for (cond, prob_dist) in attribute.items():
#     print(cond)
#     print(prob_dist)

for (cond, prob_dist) in cpd.items():
    new_fd = prob_dist.freqdist() + attribute[cond].freqdist()
    if type(prob_dist) is LaplaceProbDist:
        cpd[cond] = LaplaceProbDist(new_fd, prob_dist._bins)
        #print(cpd[cond])
    elif type(prob_dist) is MLEProbDist:
        cpd[cond] = MLEProbDist(new_fd)

tagger.__getattribute__('_transitions').update(cpd)

s
<LaplaceProbDist based on 100 samples>
q
<LaplaceProbDist based on 10 samples>
u
<LaplaceProbDist based on 51 samples>
i
<LaplaceProbDist based on 107 samples>
r
<LaplaceProbDist based on 93 samples>
e
<LaplaceProbDist based on 162 samples>
l
<LaplaceProbDist based on 67 samples>
 
<LaplaceProbDist based on 245 samples>
a
<LaplaceProbDist based on 112 samples>
m
<LaplaceProbDist based on 43 samples>
b
<LaplaceProbDist based on 8 samples>
o
<LaplaceProbDist based on 83 samples>
f
<LaplaceProbDist based on 39 samples>
t
<LaplaceProbDist based on 95 samples>
h
<LaplaceProbDist based on 60 samples>
y
<LaplaceProbDist based on 20 samples>
c
<LaplaceProbDist based on 40 samples>
d
<LaplaceProbDist based on 55 samples>
,
<LaplaceProbDist based on 23 samples>
n
<LaplaceProbDist based on 96 samples>
z
<LaplaceProbDist based on 2 samples>
g
<LaplaceProbDist based on 26 samples>
p
<LaplaceProbDist based on 31 samples>
k
<LaplaceProbDist based on 5 samples>
.
<LaplaceProbDist based on 6 samples>

In [109]:
file1 = open("a2data/cipher3/test_plain.txt","r")
file1_test_plain = file1.readlines()
file1_test_plain = [line.lower().strip('\n') for line in file1_test_plain]
file1_test_plain = [list(line) for line in file1_test_plain]
file1.close()

file1 = open("a2data/cipher3/test_cipher.txt","r")
file1_test_cipher = file1.readlines()
file1_test_cipher = [line.lower().strip('\n') for line in file1_test_cipher]
#file1_test_cipher = [list(line) for line in file1_test_cipher]
file1.close()

# #print(file1_test_cipher)
# #tagger.tag
# tagged_result = []
# # for i in range(len(file1_test_cipher)):
# #     tagged_tuple = tagger.tag(file1_test_cipher[i])
# #     tagged_list = []
# #     for tup in tagged_tuple:
# #         tagged_list.append(tup[1])
# #     tagged_result.append(tagged_list)

tagged_result = []
for i in range(len(file1_test_cipher)):
    tagged_list = tagger.tag(file1_test_cipher[i])
    tagged_result.append(tagged_list)

file1_test_plain = [list(line) for line in file1_test_plain]
#print(file1_test_plain)


#print(tagger.tag("hahah"))
#print(len(file1_test_cipher))
#print(tagged_result)
#print("__________________________")
#print((file1_test_plain))

char_counter = 0
correct_counter = 0
for i in range(len(tagged_result)):
    for j in range(len(tagged_result[i])):
        char_counter += 1
        #print(tagged_result[i][j][1],file1_test_plain[i][j])
        if tagged_result[i][j][1] == file1_test_plain[i][j]:
            correct_counter += 1

print(correct_counter)
print(char_counter)
print(correct_counter / char_counter)

246
1155
0.21298701298701297


In [ ]:

# file2 = open("a2data/cipher2/train_plain.txt","r")
# file2_train_plain = file2.readlines()
# file2_train_plain = [line.lower().strip('\n') for line in file2_train_plain]
# file2_train_plain = [list(line) for line in file2_train_plain]
# file2.close()

# file3 = open("a2data/cipher3/train_plain.txt","r")
# file3_train_plain = file3.readlines()
# file3_train_plain = [line.lower().strip('\n') for line in file3_train_plain]
# file3_train_plain = [list(line) for line in file3_train_plain]
# file3.close()

# file2 = open("a2data/cipher2/test_plain.txt","r")
# file2_test_plain = file2.readlines()
# file2_test_plain = [line.lower().strip('\n') for line in file2_test_plain]
# file2_test_plain = [list(line) for line in file2_test_plain]
# file2.close()

# file3 = open("a2data/cipher3/test_plain.txt","r")
# file3_test_plain = file3.readlines()
# file3_test_plain = [line.lower().strip('\n') for line in file3_test_plain]
# file3_test_plain = [list(line) for line in file3_test_plain]
# file3.close()

# print(file1_train_list)

In [56]:
# import numpy as np
# occurrences = np.zeros((29,29),dtype=int)
# # getting the bigram counts
# for line in new_corpus:
#     if len(line)<2:
#         continue
#     else:
#         for i in range(len(line)-1):
#             occurrences[char_dict[line[i]]][char_dict[line[i+1]]] += 1
            
# probabilities = np.zeros((29,29),dtype=float)
# for i in range(probabilities.shape[0]):
#     for j in range(probabilities.shape[1]):
#         probabilities[i][j] = occurrences[i][j] / np.sum(occurrences[i])

[[  32  591  974  799   72  216  443  124  787   20  299 2516  803 4840
    11  436   34 2699 1837 3299  321  570  136   19  606   40 1624  173
    97]
 [ 513   48   13    4 1318    1    1    3  331   44    0  470    5    7
   538    1    0  378  122   19  483   16   17    0  379    0   47    8
     9]
 [1321    2  139   11 1262    8    2 1193  586    1  371  367    2    3
  1555    0    8  344   54  805  398    3    2    0  140    2  303   28
    47]
 [ 451   17   14  147 1803   21   40   50  978    9    3   90   30   21
   566    6    0  248  309   26  307   23   41    0  143    1 4831  281
   251]
 [1869  114  905 2498  975  288  316   86  531   20   46 1362  858 3110
   233  442   91 4880 2959 1134  102  539  314  543  453   17 9666  629
   472]
 [ 413   12    5    2  610  283    7    1  472    0    2  186    4    2
   983    1    0  659   17  191  237    0    2    0   26    0 1943   38
    38]
 [ 391    3    4   21  879   40   81  643  412    0    0  161   20  120
   428    9    1